# Getting Started with Milky Way-est Analysis

*last updated -- july 2024*

*written by -- deveshi buch*

*paper authors -- deveshi buch, ethan o. nadler, risa h. wechsler, yao-yuan mao*

Welcome, and thanks for your interest in using the Milky Way-est suite of dark matter halos! As a reminder, this suite contains Milky Way-like host halos selected to have an LMC-like subhalo at present and a GSE-like subhalo that merged with it billions of years ago.

For more information on Milky Way-est, please see the paper, Buch et al. 2024. If you have questions about this notebook, you can contact the authors.

This notebook will show you how to load information from the halo catalogs (i.e., the `hlists` and `trees` files).

And a note before getting started: `mwest` will be often used to refer to the suite!

## Step 0: Data access

This notebook assumes that you have access to the Milky Way-est dataset.

**Important:** The data that this notebook uses are the halo catalog files (called `hlists` and `trees`) directly rather than the `.tar` files that are `symlib` compatible.

Once you gain access to these files, you will want to download them to your preferred location. You can use whichever method of downloading you prefer, and you can consult the Data Access Instructions page for download methods (these are written for `symlib` compatible files but you can mirror the `rclone` approach for Milky Way-est).

## Step 1: Setting up

Firstly, if you're reading this, you likely have access to Python and Jupyter. The next step is making sure you have [Yao-Yuan Mao's `helpers`](https://bitbucket.org/yymao/helpers/src/master/), which we use to load the halo catalogs.

Uncomment the line below to install it, or paste it (without the `!` at the beginning) into your terminal.

In [ ]:
#!pip install https://bitbucket.org/yymao/helpers/get/master.zip

It would also be a good idea to have packages like NumPy, SciPy, Pandas, Seaborn, Matplotlib, etc. loaded on your machine too, if you don't already! In particular, we use [NumPy](https://numpy.org/install/), [Pandas](https://pandas.pydata.org/docs/getting_started/install.html), and [Matplotlib](https://matplotlib.org/stable/install/index.html) in this notebook.

Below, we import a few scripts with some useful functions from `helpers`, define the Hubble parameter `h`, and set up our file paths.

In [ ]:
from helpers.SimulationAnalysis import SimulationAnalysis, readHlist, a2z
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os

h = 0.7  # note: don't use the variable name h anywhere else!

#######
# TODO: update this path to point the location that you have the data downloaded to
######

MAIN_ZOOMIN_DIR = '/path/to/where/your/directory/of/mwest/halos/is/downloaded'  # TODO: fill this in


In [ ]:
# get halo information
halo_info_df = pd.read_csv('mwest_halo_info.csv')

halo_names = list(halo_info_df['halo_name'])
host_analysis_timestep_str_arr = list(halo_info_df['snapshot_to_analyze'])  # z \approx 0 as described in Buch et al 2024 
host_tree_root_id_arr = list(halo_info_df['host_tree_root_id'])  # treerootid of GSE
lmc_tree_root_id_arr = list(halo_info_df['lmc_tree_root_id'])  # treerootid of LMC
gse_disrupt_id_arr = list(halo_info_df['gse_disrupt_id'])  # disruption timestep of GSE

mwest_host_treerootids = {key : val for (key,val) in zip(halo_names, host_tree_root_id_arr)}
mwest_lmc_treerootids = {key : val for (key,val) in zip(halo_names, lmc_tree_root_id_arr)}
mwest_gse_disrupt_ids = {key : val for (key,val) in zip(halo_names, gse_disrupt_id_arr)}
mwest_analysis_timesteps_str = {key : val for (key,val) in zip(halo_names, host_analysis_timestep_str_arr)}


## Step 2: Loading data

### Main branches

Let's go ahead and load the main branches of our halos (and their LMCs). The main branches trace each of the Milky Way-est halos back in time, for each timestep the halo exists, and the halo catalogs contain information about each halo at each of these timesteps. You can use this information in any analysis you want to do.

In [ ]:
# note: this cell will take a few minutes to run!

mwest_host_mbs = {}  # dictionary where the key will be halo name and the value will be its main branch
mwest_lmc_mbs = {}

for this_zoomin in halo_names:
    
    zoomin_dir = os.path.join(MAIN_ZOOMIN_DIR, this_zoomin)

    hlist_dir = os.path.join(zoomin_dir, 'hlists/')
    trees_dir = os.path.join(zoomin_dir, 'trees/')
    
    resim = SimulationAnalysis(hlist_dir, trees_dir)
    mwest_host_mbs[this_zoomin] = resim.load_main_branch(mwest_host_treerootids[this_zoomin])
    mwest_lmc_mbs[this_zoomin] = resim.load_main_branch(mwest_lmc_treerootids[this_zoomin])

print(mwest_host_mbs['Halo004'].dtype.names)
print(mwest_host_mbs['Halo004'])


The main branch of Halo004, as can be seen above, has many fields, and we can directly use them for analysis. Note that the `id` of the halo changes at each timestep. Here's the plot of the Halo004 host mass accretion history (MAH) for example:

In [ ]:
plt.plot(mwest_host_mbs['Halo004']['scale'], mwest_host_mbs['Halo004']['mvir'] / h)
plt.xlabel('scale factor a')
plt.ylabel('virial mass (Msun)')
plt.title('mass accretion history')
plt.show()


**Important notes on units:**

- Check out the ROCKSTAR documentation for detailed information: https://bitbucket.org/gfcstanford/rockstar/src/main/README.pdf
- Distances are comoving (not physical), so if you want to use the virial radius Rvir, for example, you'll want to divide it by the scale factor at which you're analyzing that Rvir.
- Many parameters have a factor of h (the Hubble parameter, which we assume to be 0.7) that needs to be removed in order to get the raw units of the parameter. For example, Mvir is the virial mass in units of Msun, but the 'mvir' field is in units of Msun/h. So, don't forget to divide by h to remove this factor!

### Halos and subhalos

Suppose we want to load all the halos at a given snapshot, or the subhalos of one of our Milky Way-est hosts. Here's how we'd do that:

In [ ]:
# example using Halo004, at its analysis timestep (which happens to be right at z=0 or a=1)

zoomin_of_interest = 'Halo004'
timestep_of_interest = float(mwest_analysis_timesteps[zoomin_of_interest].strip('"'))

print('zoomin of interest: ' + str(zoomin_of_interest))
print('timestep of interest: ' + str(timestep_of_interest))

zoomin_dir = os.path.join(MAIN_ZOOMIN_DIR, zoomin_of_interest)

hlist_dir = os.path.join(zoomin_dir, 'hlists/')
trees_dir = os.path.join(zoomin_dir, 'trees/')

# get all halos
resim = SimulationAnalysis(hlist_dir, trees_dir)
all_halos_at_this_snapshot = resim.load_halos(z=a2z(timestep_of_interest), additional_fields=['vacc', 'mpeak', 'macc'])
print("number of halos found: " + str(len(all_halos_at_this_snapshot)))

# get only halos that are large enough given the resolution limit
halos_above_resolution_limit = all_halos_at_this_snapshot[np.where(all_halos_at_this_snapshot['mvir']/h > 1.2e8)[0]]
print("number of halos above the resolution limit: " + str(len(halos_above_resolution_limit)))

# get subhalos of a given halo
timestep_of_interest_index = np.argmin(np.abs(mwest_host_mbs[zoomin_of_interest]['scale'] - timestep_of_interest))
zoomin_of_interest_id = mwest_host_mbs[zoomin_of_interest][timestep_of_interest_index]['id']
subhalos_of_zoomin_at_this_timestep = halos_above_resolution_limit[np.where(halos_above_resolution_limit['upid'] == zoomin_of_interest_id)[0]]
print("number of subhalos of zoomin of interest: " + str(len(subhalos_of_zoomin_at_this_timestep)))


**Note on resolution:** The mass of the highest-resolution particles is 4.0 x 10^5 Msun. This means that, with a 300-particle resolution limit, our subhalo analysis should only include subhalos with masses above 4.0 x 10^5 Msun x 300 = 1.2 x 10^8 Msun. If this resolution cut is applicable to you, we suggest you incorporate it into your pipeline! (Otherwise, the halos pulled directly from the halo catalogs without this additional cut will include halos with masses below this threshold.)

### Other tips

In addition to the very important notes above (which you should go back and read if you skipped over them), some things to keep in mind:

- Occasionally different ways of loading in halos will result in slightly different namings of the parameters (e.g., 'Mvir' instead of 'mvir'), so if you're ever confused about why a field seems to be missing when it should be included in your main branch, just check what the labels are!
